In [1]:
from __future__ import annotations

import ibis
from ibis import _
from ibis.expr.types import BooleanValue, Table

from mismo.block import Blocking, block_on_arrays, block
from mismo.datasets import load_patents

ibis.options.interactive = True

In [2]:
patents = load_patents()
patents

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                                          ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                                           │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                       │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                     │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                           │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                                           │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B                   │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │     0.00 │  0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  GABRIAGUES                         │ H04B**H01S**H04J                                                 │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │     0.00 │  0.000000 │ LEE  RICKLER**SIMON  PARKER**CANON RES CENT EURO **RAKEFET  SAGMAN**TIMOTHY FRA… │ G06F                                                             │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │     0.00 │  0.000000 │ QI HE  HONG**ADAM MICHAEL  BAUMBERG**ALEXANDER RALPH  LYONS                      │ G06T**G01B                                                       │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │     0.00 │  0.000000 │ NILESH  PATHAK**MASAMICHI  MASUDA** CANON TECHNOLOGY EURO **PATRICK WILLIAM  MO… │ H04B**G06T**G06F**H04M**H04N**H04Q**G03B**B41J**G01B**G06Q       │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │     0.00 │  0.000000 │ GABRIEL MARINUS  MEESTERS**RUDOLF CAROLUS  BARENDSE**ARIE KARST  KIES**ALEXANDE… │ C12N**A61K**A23L**A23J**A23K**A01H**B01J**C12R**C07D**A61P**B01D │
│         … │          … │ …                    │ …                            │        … │         … │ …                                                                                │ …  

In [3]:
from mismo.clean.strings import norm_whitespace  # noqa: E402


def clean_names(names):
    names = norm_whitespace(names)
    names = names.upper()
    # Only want to keep letters, numbers, and spaces
    names = names.re_replace("[^0-9A-Z ]", "")
    # Now have to do whitespace fixup again
    names = norm_whitespace(names)
    return names


def bin_coordinate(coord):
    """Bin coordinates to 0.1 degree precision, which is ~6 miles.

    38.5323 -> 385
    38.4923 -> 385
    """
    return (coord.round(1) * 10).cast("int16")


cleaned = patents
cleaned = cleaned.select(
    "record_id",
    "label_true",
    "name_true",
    "name",
    name_cleaned=clean_names(_.name),
    latitude=_.latitude.nullif(0),
    longitude=_.longitude.nullif(0),
    coauthors=_.coauthors.upper().split("**").map(norm_whitespace).sort(),
    classes=_.classes.upper().split("**").map(norm_whitespace).sort(),
)
cleaned = cleaned.mutate(
    name_tokens=_.name_cleaned.split(" ").map(norm_whitespace).sort(),
    name_first3=_.name_cleaned[0:3],
    latitude_binned=bin_coordinate(_.latitude),
    longitude_binned=bin_coordinate(_.longitude),
)
cleaned

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ name_cleaned             ┃ latitude ┃ longitude ┃ coauthors                                                                           ┃ classes                  ┃ name_tokens                 ┃ name_first3 ┃ latitude_binned ┃ longitude_binned ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ string                   │ float64  │ float64   │ array<string>                                                                       │ array<string>            │ array<string>               │ string      │ int16           │ int16            │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────────────────────┼──────────┼───────────┼─────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────┼─────────────────────────────┼─────────────┼─────────────────┼──────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │ AGILENT TECHNOLOGIES INC │      nan │       nan │ ['DAVID E SNYDER', 'KONINK PHILIPS ELECTRONICS N V', ... +1]                        │ ['A61B', 'A61N']         │ ['AGILENT', 'INC', ... +1]  │ AGI         │            NULL │             NULL │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │ AKZO NOBEL NV            │      nan │       nan │ ['ALBERT BORREMAN', 'ALBERTO SLIKTA', ... +11]                                      │ ['B01L', 'C11D', ... +3] │ ['AKZO', 'NOBEL', ... +1]   │ AKZ         │            NULL │             NULL │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │ AKZO NOBEL NV            │      nan │       nan │ ['HANS OSKARSSON', 'JOHANNA CHRISTINA SPEELMAN', ... +3]                            │ ['B01F', 'C09K', ... +2] │ ['AKZO', 'NOBEL', ... +1]   │ AKZ         │            NULL │             NULL │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │    52.35 │  4.916667 │ ['EVAN JOHN STANBURY', 'GUENTER KOCHSMEIER', ... +2]                                │ ['G02B', 'G04G', ... +2] │ ['ALCATEL', 'NV']           │ ALC         │             524 │               49 │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │    52.35 │  4.916667 │ ['CHRISTIAN LE SERGENT', 'DUANE LYNN MORTENSEN', ... +2]                            │ ['B05D', 'C03B', ... +6] │ ['ALCATEL', 'NV']           │ ALC         │             524 │               49 │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │      nan │       nan │ ['JEAN MICHEL GABRIAGUES', 'MICHEL SOTOM', ... +1]                                  │ ['H01S', 'H04B', ... +1] │ ['ALCATEL', 'NV']           │ ALC         │            NULL │             NULL │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │ CANON EUROPA NV          │      nan │       nan │ ['CANON RES CENT EURO', 'LEE RICKLER', ... +3]                                      │ ['G06F']                 │ ['CANON', 'EUROPA', ... +1] │ CAN         │            NULL │             NULL │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │ CANON EUROPA NV    

In [4]:
def coords_match(left: Table, right: Table) -> BooleanValue:
    return [
        (
            (left.latitude_binned == right.latitude_binned)
            & (left.longitude_binned == right.longitude_binned)
        ),
        (
            (left.latitude_binned == right.latitude_binned)
            & left.longitude_binned.isnull()
            & right.longitude_binned.isnull()
        ),
        (
            left.latitude_binned.isnull()
            & right.latitude_binned.isnull()
            & (left.longitude_binned == right.longitude_binned)
        ),
    ]


rules = [
    coords_match,
    lambda left, right: left.name_cleaned == right.name_cleaned,
    block_on_arrays("coauthors", "classes"),
]


cleaned = cleaned.cache()
left = cleaned
right = cleaned.view()
blocking = block(left, right, rules)
blocked = blocking.blocked.cache()
blocked

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                ┃ classes_r        ┃ coauthors_l                                                                   ┃ coauthors_r                                                               ┃ label_true_l ┃ label_true_r ┃ latitude_binned_l ┃ latitude_binned_r ┃ latitude_l ┃ latitude_r ┃ longitude_binned_l ┃ longitude_binned_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                      ┃ name_cleaned_r                                       ┃ name_first3_l ┃ name_first3_r ┃ name_l                                 ┃ name_r                                                ┃ name_tokens_l                          ┃ name_tokens_r                            ┃ name_true_l                  ┃ name_true_r                                 ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ array<string>            │ array<string>    │ array<string>                                                                 │ array<string>                                                             │ int64        │ int64        │ int16             │ int16             │ float64    │ float64    │ int16              │ int16              │ float64     │ float64     │ string                              │ string                                               │ string        │ string        │ string                                 │ string                                                │ array<string>                          │ array<string>                            │ string                       │ string                                      │
├─────────────┼─────────────┼──────────────────────────┼──────────────────┼───────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────┼──────────────┼──────────────┼───────────────────┼───────────────────┼────────────┼────────────┼────────────────────┼────────────────────┼─────────────┼─────────────┼─────────────────────────────────────┼──────────────────────────────────────────────────────┼───────────────┼───────────────┼────────────────────────────────────────┼───────────────────────────────────────────────────────┼────────────────────────────────────────┼──────────────────────────────────────────┼──────────────────────────────┼─────────────────────────────────────────────┤
│    18321565 │    18321262 │ ['G01N', 'G02B', ... +2] │ ['C12M', 'C12Q'] │ ['HORIJON JOSEPH LO

In [5]:
blocking.inners

(<mismo.block._blocking.OrBlocking at 0x11dd8c5d0>,
 <mismo.block._blocking.IdsBlocking at 0x11def2fd0>)

In [6]:
blocked.count()

398527

In [7]:
from mismo.compare import (  # noqa: E402
    Comparison,
    ComparisonLevel,
    exact_level,
    jaccard,
)
from mismo.compare.fs import FellegiSunterComparer  # noqa: E402

exact = exact_level("name")
almost_level = exact_level("name_first3")
name_comparison = Comparison(name="Name", levels=[exact, almost_level])

classes_exact_match = exact_level("classes")
classes_jaccard_50 = ComparisonLevel(
    name="Classes 50% Jaccard",
    condition=lambda table: jaccard(table.classes_l, table.classes_r) >= 0.5,
)
classes_comparison = Comparison(
    name="Classes",
    levels=[classes_exact_match, classes_jaccard_50],
)

fs = FellegiSunterComparer([name_comparison, classes_comparison], weights=None)
fs.comparisons

[Comparison(name=Name, levels=[ComparisonLevel(name=exact_name, description=Exact match on `name`), ComparisonLevel(name=exact_name_first3, description=Exact match on `name_first3`)]),
 Comparison(name=Classes, levels=[ComparisonLevel(name=exact_classes, description=Exact match on `classes`), ComparisonLevel(name=Classes 50% Jaccard)])]

In [8]:
fs_trained = fs.trained(left, right, max_pairs=10_000, seed=42)
fs_trained.weights.prior = 3 / left.count().execute()
fs_trained

FellegiSunterComparer([Comparison(name=Name, levels=[ComparisonLevel(name=exact_name, description=Exact match on `name`), ComparisonLevel(name=exact_name_first3, description=Exact match on `name_first3`)]), Comparison(name=Classes, levels=[ComparisonLevel(name=exact_classes, description=Exact match on `classes`), ComparisonLevel(name=Classes 50% Jaccard)])], <mismo.compare.fs._base.Weights object at 0x11e030bd0>)

In [9]:
comparisons = fs_trained.compare(blocked)
comparisons

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                ┃ classes_r        ┃ coauthors_l                                                                   ┃ coauthors_r                                                               ┃ label_true_l ┃ label_true_r ┃ latitude_binned_l ┃ latitude_binned_r ┃ latitude_l ┃ latitude_r ┃ longitude_binned_l ┃ longitude_binned_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                      ┃ name_cleaned_r                                       ┃ name_first3_l ┃ name_first3_r ┃ name_l                                 ┃ name_r                                                ┃ name_tokens_l                          ┃ name_tokens_r                            ┃ name_true_l                  ┃ name_true_r                                 ┃ Name_cmp          ┃ Name_bf  ┃ Classes_cmp ┃ Classes_bf ┃ bf       ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ int64       │ int64       │ array<string>            │ array<string>    │ array<string>                                                                 │ array<string>                                                             │ int64        │ int64        │ int16             │ int16             │ float64    │ float64    │ int16              │ int16              │ float64     │ float64     │ string                              │ string                                               │ string        │ string        │ string                                 │ string                                                │ array<string>                          │ array<string>                            │ string                       │ string                                      │ string            │ float64  │ string      │ float64    │ float64  │
├─────────────┼─────────────┼──────────────────────────┼──────────────────┼───────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────┼──────────────┼──────────────┼───────────────────┼───────────────────┼────────────┼────────────┼────────────────────┼────────────────────┼─────────────┼─────────────┼─────────────────────────────────────┼──────────────────────────────────────────────────────┼───────────────┼───────────────┼────────────────────────────────────────┼─────────────────────────────────────

Look at the different combinations of comparisons that we found, and see which ones
led to the best match, and which ones to the worst.

Unsurprisingly, the exact match levels have the highest bayes factor, and the
ELSE levels have the lowest. The other levels are somewhere in between.

In [10]:
comparisons.distinct(on=["Name_cmp", "Classes_cmp"]).order_by("bf")

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                 ┃ classes_r                 ┃ coauthors_l                                           ┃ coauthors_r                                                 ┃ label_true_l ┃ label_true_r ┃ latitude_binned_l ┃ latitude_binned_r ┃ latitude_l ┃ latitude_r ┃ longitude_binned_l ┃ longitude_binned_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                                     ┃ name_cleaned_r                                     ┃ name_first3_l ┃ name_first3_r ┃ name_l                                                ┃ name_r                                                 ┃ name_tokens_l                          ┃ name_tokens_r                          ┃ name_true_l                                     ┃ name_true_r                                     ┃ Name_cmp          ┃ Name_bf   ┃ Classes_cmp         ┃ Classes_bf ┃ bf       ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ int64       │ int64       │ array<string>             │ array<string>             │ array<string>                                         │ array<string>                                               │ int64        │ int64        │ int16             │ int16             │ float64    │ float64    │ int16              │ int16              │ float64     │ float64     │ string                                             │ string                                             │ string        │ string        │ string                                                │ string                                                 │ array<string>                          │ array<string>                          │ string                                          │ string                                          │ string            │ float64   │ string              │ float64    │ float64  │
├─────────────┼─────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────────────────────────────────┼─────────────────────────────────────────────────────────────┼──────────────┼──────────────┼───────────────────┼───────────────────┼────────────┼────────────┼────────────────────┼────────────────────┼─────────────┼─────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼─

Let's be really picky and only take the most likely matches as true matches, and
then perform connected components to label each patent with its inventor:

In [11]:
links = comparisons[_.bf == _.bf.max()]
links = links.cache()
links.count()

2656

In [12]:
from mismo.cluster import connected_components  # noqa: F401

labels = connected_components(links)
labels = labels.relabel({"component": "inventor_id"})
labels

┏━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ record_id ┃ inventor_id ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64     │ int64       │
├───────────┼─────────────┤
│  35244194 │    35244194 │
│    578490 │      578490 │
│    578471 │      578471 │
│  18321547 │    18321547 │
│  18321573 │    18321573 │
│    578474 │      578474 │
│  18321593 │    18321593 │
│  18321566 │    18321566 │
│  18321582 │    18321582 │
│  18321543 │    18321543 │
│         … │           … │
└───────────┴─────────────┘

In [13]:
with_labels = patents.left_join(labels, "record_id")
with_labels

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                                          ┃ record_id_right ┃ inventor_id ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                                           │ int64           │ int64       │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┼─────────────────┼─────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                       │            2909 │        2909 │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                     │            3574 │        3574 │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                           │            3575 │        3575 │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                                           │            3779 │        3779 │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B                   │            3780 │        3780 │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │     0.00 │  0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  GABRIAGUES                         │ H04B**H01S**H04J                                                 │            3782 │        3782 │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │     0.00 │  0.000000 │ LEE  RICKLER**SIMON  PARKER**CANON RES CENT EURO **RAKEFET  SAGMAN**TIMOTHY FRA… │ G06F                                                             │           15041 │       15041 │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │     0.00 │  0.000000 │ QI HE  HONG**ADAM MICHAEL  BAUMBERG**ALEXANDER RALPH  LYONS                      │ G06T**G01B                                                       │           15042 │       15042 │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │     0.00 │  0.000000 │ NILESH  PATHAK**MASAMICHI  MASUDA** CANON TECHNOLOGY EURO **PATRICK WILLIAM  MO… │ H04B**G06T**G06F**H04M**H04N**H04Q**G03B**B41J**G01B**G06Q       │           15043 │       1504

In [14]:
# Some records don't appear in the labeling because they didn't appear in the links.
# connected_components() should be updated to so you can supply nodes as well,
# so every node gets a component.
with_labels[_.inventor_id.isnull()]

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true                      ┃ name                                           ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes ┃ record_id_right ┃ inventor_id ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64     │ int64      │ string                         │ string                                         │ float64  │ float64   │ string                                                                           │ string  │ int64           │ int64       │
├───────────┼────────────┼────────────────────────────────┼────────────────────────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼─────────┼─────────────────┼─────────────┤
│    317966 │     319886 │ ADIDAS INTERNAT MARKETING      │ Adidas International Marketing B.V.            │    52.35 │  4.916667 │ RENNER PHILIPP**VAN OORSCHOT JANNEKE**UEDELHOVEN JUERGEN**WHITEMAN JOHN**NIEDNE… │ NULL    │            NULL │        NULL │
│    317977 │     319886 │ ADIDAS INTERNAT MARKETING      │ adidas International Marketing BV              │    52.35 │  4.916667 │ LUSSIER MICHEL                                                                   │ NULL    │            NULL │        NULL │
│  15313541 │   15350254 │ JANSSEN PHARM                  │ JANSSEN PHARMACEUTICA, N.V.                    │     0.00 │  0.000000 │ HARALD KALLABIS**BAYER SCHERING PHARMA AKTIENGES**SUSANNE ROHRIG**THEODOURE SPI… │ NULL    │            NULL │        NULL │
│  18319895 │   26842439 │ ELECTRONICS KONINKIJKE PHILIPS │ Koninkijke Philips Electronics, N.V.           │    51.45 │  5.466667 │ KANG KEITH TAN CHEE**KOK TERENCE LOH TZE                                         │ NULL    │            NULL │        NULL │
│  18321287 │   26842439 │ ELECTRONCIS KONINK PHILIPS     │ Koninklijke Philips Electroncis N. V.          │    51.45 │  5.466667 │ BEEDHAM MARTYN                                                                   │ NULL    │            NULL │        NULL │
│  18321362 │   26842439 │ ELECTRONICS KONINK PHILIPS     │ Koninklijke Philips Electronics                │    51.45 │  5.466667 │ GREENE DAN**LAMASON SIMON**BLACK CRAIG**STOEFFLER FRIEDRICH                      │ NULL    │            NULL │        NULL │
│  18321399 │   26842439 │ ELECTRONICS KONINK PHILIPS     │ Koninklijke Philips Electronics N. V.          │    52.50 │  5.750000 │ YANG TAH CHING                                                                   │ NULL    │            NULL │        NULL │
│  26343943 │   26358567 │ PAUL WURTH                     │ PAUL WURTH S.A.                                │     0.00 │  0.000000 │ PLEIMELDING CLAUDE**MOUSEL NICOLAS**MAGGIOLI NICOLAS                             │ NULL    │            NULL │        NULL │
│  30956705 │   30982984 │ INTE MIJ RES SHELL             │ SHELL INTERNACIONALE RESEARCH MAATSCHAPPIJ B.V │     0.00 │  0.000000 │ BOSMA MARTIN GERARD RENE**GEILIKMAN MIKHAIL BORIS**CORNELISSEN ERIK KERST        │ NULL    │            NULL │        NULL │
│  32288244 │   32302628 │ JUDE MEDICAL PUERTO RICO ST    │ ST. JUDE MEDICAL PUERTO RICO LLC               │     0.00 │  0.000000 │ FORSBERG ANDREW THOMAS**WHITE JOHN                                               │ NULL    │            NULL │        NULL │
│         … │          … │ …                              │ …                                              │        … │         … │ …            

In [15]:
links[(_.record_id_l == 317966) | (_.record_id_r == 317966)]

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l     ┃ classes_r     ┃ coauthors_l   ┃ coauthors_r   ┃ label_true_l ┃ label_true_r ┃ latitude_binned_l ┃ latitude_binned_r ┃ latitude_l ┃ latitude_r ┃ longitude_binned_l ┃ longitude_binned_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l ┃ name_cleaned_r ┃ name_first3_l ┃ name_first3_r ┃ name_l ┃ name_r ┃ name_tokens_l ┃ name_tokens_r ┃ name_true_l ┃ name_true_r ┃ Name_cmp ┃ Name_bf ┃ Classes_cmp ┃ Classes_bf ┃ bf      ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━┩
│ int64       │ int64       │ array<string> │ array<string> │ array<string> │ array<string> │ int64        │ int64        │ int16             │ int16             │ float64    │ float64    │ int16              │ int16              │ float64     │ float64     │ string         │ string         │ string        │ string        │ string │ string │ array<string> │ array<string> │ string      │ string      │ string   │ float64 │ string      │ float64    │ float64 │
└─────────────┴─────────────┴───────────────┴───────────────┴───────────────┴───────────────┴──────────────┴──────────────┴───────────────────┴───────────────────┴────────────┴────────────┴────────────────────┴────────────────────┴─────────────┴─────────────┴────────────────┴────────────────┴───────────────┴───────────────┴────────┴────────┴───────────────┴───────────────┴─────────────┴─────────────┴──────────┴─────────┴─────────────┴────────────┴─────────┘